In [1]:
import json
import urllib.request
import dill
import base64
import types
from urllib.error import HTTPError
import ast
import types

# Lambda endpoints
grader_api_url = 'https://wyv616tp17.execute-api.us-east-1.amazonaws.com/default/Grader'
grader_api_key = 'Kd32fl3g3p917iM0zwjiO23Bitj4PO9ga4LektOa'
grades_api_url = 'https://1rwoprdby6.execute-api.us-east-1.amazonaws.com/default/Grades'
grades_api_key = 'lY1O5NDRML9zEyRvWhf0c1GeEYFe3BE710Olbh3R'

# Request types
STUDENT_GRADE_REQUEST = 'STUDENT_GRADE'


class PennGrader:
    
    def __init__(self, homework_id, student_id):
        if '_' in str(student_id):
            raise Exception("Student ID cannot contain '_'")
        self.homework_id = homework_id
        self.student_id = str(student_id)

        
    def grade(self, test_case_id, answer):
        request = { 
            'homework_id' : self.homework_id, 
            'student_id' : self.student_id, 
            'test_case_id' : test_case_id,
            'answer' : self._serialize(answer)
        }
        response = self._send_request(request, grader_api_url, grader_api_key)
        print(response)
    
    
    def view_score(self):
        request = { 
            'homework_id' : self.homework_id, 
            'student_id' : self.student_id, 
            'request_type' : STUDENT_GRADE_REQUEST,
        }
        response = self._send_request(request, grades_api_url, grades_api_key)
        if 'Error' in response:
            print(response)
            return None
        else:
            grades, deadline, max_daily_submissions, max_score = self._deserialize(response)
            grades_df = pd.DataFrame(grades)
            
            # Extract student ID from [student_submission_id]
            grades_df['student_id'] = grades_df['student_submission_id'].apply(lambda x: str(x).split('_')[0])
            grades_df['test_case_id'] = grades_df['student_submission_id'].apply(lambda x: '_'.join(str(x).split('_')[1:]))

            # Convert to correct types
            grades_df['student_score'] = grades_df['student_score'].astype(int)
            
            display(grades_df[['student_id', 'test_case_id', 'student_score','max_score']])
            print('Total Score: {}/{}'.format(grades_df['student_score'].sum(), max_score))
            print('Deadline: {}'.format(deadline))
            print('Max daily submission per test case: {}'.format(max_daily_submissions))
            print('\nRemember: If you run any test case after the deadline your homework will be marked late.')
            print('\nAlso, make sure your student ID is correct.')
            print('\nAlso, you are a star!')

    
    def _send_request(self, request, api_url, api_key):
        params = json.dumps(request).encode('utf-8')
        headers = {'content-type': 'application/json', 'x-api-key': api_key}
        request = urllib.request.Request(api_url, data=params, headers=headers)
        try:
            response = urllib.request.urlopen(request)
            return '{}'.format(response.read().decode('utf-8'))
        except HTTPError as error:
            return 'Error: {}'.format(error.read().decode("utf-8")) 
        

    def _serialize(self, obj):
        byte_serialized = dill.dumps(obj, recurse = True)
        return base64.b64encode(byte_serialized).decode("utf-8")
    
    def _deserialize(self, obj):
        byte_decoded = base64.b64decode(obj)
        return dill.loads(byte_decoded)

## Autograder Setup
Enter your 8-digit PennID below:

In [2]:
STUDENT_ID = 99999999

Run the following cell to initialize the autograder.

**NOTE:** Remember we store your submissions and check against other student's submissions... so, not that you would, but no cheating.

In [3]:
grader = PennGrader(homework_id = 'CIS545_Spring_2019_HW1', student_id = STUDENT_ID)

## Question 1: Write an addition function!
Fill in the function body of `addition_function` that takes in two numbers and returns their sum.

In [20]:
def addition_function(x,y):
    return x + y

Run the following cell to grade this question.

In [21]:
# Autograder Cell: Test Case #1
grader.grade(test_case_id = 'test_case_1', answer = addition_function)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Question 2:  Now write a Class!
Add two class functions: `add_course` and `remove_course` that both take in a string representing a course ID.

In [4]:
class Student:
    def __init__(self, name = 'John', email = 'john@email', courses = []):
        self.name = name
        self.email = email
        self.courses = courses 

    def get_coures(self):
        return self.courses 

    def add_course(self, course):
        self.courses.append(course)
    
    def remove_course(self, course):
        self.courses.remove(course)

In [5]:
# Autograder cell: Test case #2
grader.grade(test_case_id = 'test_case_2', answer = Student)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Question 3:  Finally, let's do some Pandas stuff!

Run the following cell to load the dataset...

In [ ]:
import pandas as pd

students_df = pd.DataFrame([['Leonardo_Murri',100],['Akshay_Grewal',65]], columns = ['first_last_name','grade'])
students_df

Use `.apply(...)` to keep only the last name in the `first_last_name` column.

In [ ]:
students_df['first_last_name'] = students_df['first_last_name'].apply(lambda x: x.split('_')[0])
students_df

Now, rename the `first_last_name` column to `first_name`. Make sure your final dataframe is called `student_df`.

In [ ]:
students_df = students_df.rename(columns = {'first_last_name':'first_name'})
students_df

Run the following cell to grade this question.

In [ ]:
# Autograder cell: Test case #3
grader.grade(test_case_id = 'test_case_3', answer = students_df)

## View your score!
You made it this far, run the following cell to double check we stored all your scores correctly.

In [ ]:
grader.view_score() 